###Descripción del Dataset PIMA DIABETES


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [58]:
diabetes = pd.read_csv('diabetes.csv')
print(diabetes.shape)

(768, 9)


In [59]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
diabetes.head()

In [ ]:
diabetes.isnull().sum()

###Análisis Exploratorio de Datos

In [ ]:
import seaborn as sns
sns.pairplot(diabetes, hue = 'Outcome', height= 2, palette='colorblind');

In [ ]:
#Para entender el comportamiento de la gráfica de pares
import seaborn as sns
sns.pairplot(diabetes, hue = 'Pregnancies', height= 2, palette='colorblind');

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(diabetes.corr(), annot= True, square=True,ax=ax);

###Preparación de los datos


In [60]:
#Separación entre entrenamiento y test
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, stratify = diabetes['Outcome'], test_size = 0.3, random_state = 100)

print(train.shape, test.shape)

(537, 9) (231, 9)


In [61]:
#Separación de los conjuntos de datos
X_train = np.c_[train['Pregnancies'], train['Glucose'],train['BloodPressure'],train['SkinThickness'],train['Insulin'],train['BMI'],train['DiabetesPedigreeFunction'],train['Age']]
y_train = train['Outcome'].values

X_test = np.c_[test['Pregnancies'], test['Glucose'],test['BloodPressure'],test['SkinThickness'],test['Insulin'],test['BMI'],test['DiabetesPedigreeFunction'],test['Age']]
y_test = test['Outcome'].values

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(537, 8) (537,)
(231, 8) (231,)


###Modelo con  Scikit-Learn

In [62]:
#Definición del modelo
from sklearn.neural_network import MLPClassifier

red_skl = MLPClassifier(hidden_layer_sizes = (64,32,16,8,4,2,1), activation = 'logistic', solver = 'sgd', max_iter = 100000000)
red_skl.fit(X_train, y_train)

MLPClassifier(activation='logistic',
              hidden_layer_sizes=(64, 32, 16, 8, 4, 2, 1), max_iter=100000000,
              solver='sgd')

In [63]:
#evaluación
from sklearn.metrics import classification_report, confusion_matrix

y_train_pred = red_skl.predict(X_train)
y_test_pred = red_skl.predict(X_test)

print("Rendimiento en Entrenamiento")
print(confusion_matrix(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))


print("Rendimiento en Testeo")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

Rendimiento en Entrenamiento
[[350   0]
 [187   0]]
              precision    recall  f1-score   support

           0       0.65      1.00      0.79       350
           1       0.00      0.00      0.00       187

    accuracy                           0.65       537
   macro avg       0.33      0.50      0.39       537
weighted avg       0.42      0.65      0.51       537

Rendimiento en Testeo
[[150   0]
 [ 81   0]]
              precision    recall  f1-score   support

           0       0.65      1.00      0.79       150
           1       0.00      0.00      0.00        81

    accuracy                           0.65       231
   macro avg       0.32      0.50      0.39       231
weighted avg       0.42      0.65      0.51       231



/Users/joelsebastiantorrescarrasco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joelsebastiantorrescarrasco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/joelsebastiantorrescarrasco/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

###Crear Modelo con PyTorch

In [64]:
#Importar los módulos

import torch
import torch.nn as nn
import torch.nn.functional as F

X_train_t = torch.FloatTensor(X_train)
X_test_t = torch.FloatTensor(X_test)

y_train_t = torch.LongTensor(y_train)
y_test_t = torch.LongTensor(y_test)

In [90]:
#Crear modelo
class red_torch(nn.Module):
  def __init__(self, input_features = 8, hidden1 = 16, hidden2 = 32, hidden3 = 16, out_features = 2):
    super().__init__()
    self.f_conected1 = nn.Linear(input_features, hidden1)
    self.f_conected2 = nn.Linear(hidden1, hidden2)
    self.f_conected3 = nn.Linear(hidden2, hidden3)
    self.out = nn.Linear(hidden3, out_features)

  def forward(self, x):
    x = F.relu(self.f_conected1(x))
    x = F.relu(self.f_conected2(x))
    x = F.relu(self.f_conected3(x))
    x = self.out(x)
    return x

In [91]:
#instanciación del modelo
torch.manual_seed(100)
red_pytorch = red_torch()
red_pytorch.parameters()

<generator object Module.parameters at 0x32274f5a0>

In [95]:
#Entrenamiento
costo = nn.CrossEntropyLoss()
optimizador = torch.optim.SGD(red_pytorch.parameters(), lr = 0.001)
epochs = 200000
costos_totales = []

for i in range(epochs):
  y_pred_t = red_pytorch.forward(X_train_t)
  c = costo(y_pred_t, y_train_t)
  costos_totales.append(c)

  if i%10 == 1:
    print('Generacion: {} con costo: {}'.format(i,c.item()))

  optimizador.zero_grad()
  c.backward()
  optimizador.step()


Generacion: 1 con costo: 0.4735722541809082
Generacion: 11 con costo: 0.4756196439266205
Generacion: 21 con costo: 0.4767427146434784
Generacion: 31 con costo: 0.4773607850074768
Generacion: 41 con costo: 0.4772147238254547
Generacion: 51 con costo: 0.47649264335632324
Generacion: 61 con costo: 0.4759295880794525
Generacion: 71 con costo: 0.4753122925758362
Generacion: 81 con costo: 0.47493985295295715
Generacion: 91 con costo: 0.4747526943683624
Generacion: 101 con costo: 0.4747217893600464
Generacion: 111 con costo: 0.4739322066307068
Generacion: 121 con costo: 0.47295162081718445
Generacion: 131 con costo: 0.4728182852268219
Generacion: 141 con costo: 0.4732505679130554
Generacion: 151 con costo: 0.4741086959838867
Generacion: 161 con costo: 0.47499579191207886
Generacion: 171 con costo: 0.4754904508590698
Generacion: 181 con costo: 0.4758032262325287
Generacion: 191 con costo: 0.47565364837646484
Generacion: 201 con costo: 0.4753168523311615
Generacion: 211 con costo: 0.47504675388

Generacion: 2331 con costo: 0.4666889011859894
Generacion: 2341 con costo: 0.46658849716186523
Generacion: 2351 con costo: 0.466621071100235
Generacion: 2361 con costo: 0.4663233160972595
Generacion: 2371 con costo: 0.4663311243057251
Generacion: 2381 con costo: 0.46613478660583496
Generacion: 2391 con costo: 0.4661341607570648
Generacion: 2401 con costo: 0.465568870306015
Generacion: 2411 con costo: 0.4654836356639862
Generacion: 2421 con costo: 0.4654160439968109
Generacion: 2431 con costo: 0.4652767777442932
Generacion: 2441 con costo: 0.46569451689720154
Generacion: 2451 con costo: 0.46594977378845215
Generacion: 2461 con costo: 0.46634459495544434
Generacion: 2471 con costo: 0.46653494238853455
Generacion: 2481 con costo: 0.4665779769420624
Generacion: 2491 con costo: 0.4664311707019806
Generacion: 2501 con costo: 0.466258704662323
Generacion: 2511 con costo: 0.4661853313446045
Generacion: 2521 con costo: 0.46610745787620544
Generacion: 2531 con costo: 0.46590203046798706
Generaci

Generacion: 4071 con costo: 0.4619119465351105
Generacion: 4081 con costo: 0.4617283344268799
Generacion: 4091 con costo: 0.4614357352256775
Generacion: 4101 con costo: 0.46051856875419617
Generacion: 4111 con costo: 0.4597250521183014
Generacion: 4121 con costo: 0.45657309889793396
Generacion: 4131 con costo: 0.45427370071411133
Generacion: 4141 con costo: 0.45373594760894775
Generacion: 4151 con costo: 0.4537656009197235
Generacion: 4161 con costo: 0.45441359281539917
Generacion: 4171 con costo: 0.45513924956321716
Generacion: 4181 con costo: 0.4552076756954193
Generacion: 4191 con costo: 0.4556703269481659
Generacion: 4201 con costo: 0.45714426040649414
Generacion: 4211 con costo: 0.4633708894252777
Generacion: 4221 con costo: 0.46771135926246643
Generacion: 4231 con costo: 0.4666248857975006
Generacion: 4241 con costo: 0.4654117226600647
Generacion: 4251 con costo: 0.4642185866832733
Generacion: 4261 con costo: 0.46361103653907776
Generacion: 4271 con costo: 0.4624721109867096
Gene

Generacion: 6331 con costo: 0.4545215964317322
Generacion: 6341 con costo: 0.4540216326713562
Generacion: 6351 con costo: 0.45413410663604736
Generacion: 6361 con costo: 0.4541710913181305
Generacion: 6371 con costo: 0.4545132517814636
Generacion: 6381 con costo: 0.45403361320495605
Generacion: 6391 con costo: 0.454898864030838
Generacion: 6401 con costo: 0.45284712314605713
Generacion: 6411 con costo: 0.4530417025089264
Generacion: 6421 con costo: 0.4537099301815033
Generacion: 6431 con costo: 0.45326513051986694
Generacion: 6441 con costo: 0.45278963446617126
Generacion: 6451 con costo: 0.45293930172920227
Generacion: 6461 con costo: 0.45368608832359314
Generacion: 6471 con costo: 0.4545237123966217
Generacion: 6481 con costo: 0.4544496238231659
Generacion: 6491 con costo: 0.4541202187538147
Generacion: 6501 con costo: 0.453457236289978
Generacion: 6511 con costo: 0.45330727100372314
Generacion: 6521 con costo: 0.45276010036468506
Generacion: 6531 con costo: 0.452795147895813
Generac

Generacion: 8611 con costo: 0.44857561588287354
Generacion: 8621 con costo: 0.44857117533683777
Generacion: 8631 con costo: 0.44811975955963135
Generacion: 8641 con costo: 0.4476393461227417
Generacion: 8651 con costo: 0.44801056385040283
Generacion: 8661 con costo: 0.4443762004375458
Generacion: 8671 con costo: 0.4428643584251404
Generacion: 8681 con costo: 0.44247162342071533
Generacion: 8691 con costo: 0.44264984130859375
Generacion: 8701 con costo: 0.444193571805954
Generacion: 8711 con costo: 0.44855621457099915
Generacion: 8721 con costo: 0.45053789019584656
Generacion: 8731 con costo: 0.4499865770339966
Generacion: 8741 con costo: 0.4494098722934723
Generacion: 8751 con costo: 0.4488908052444458
Generacion: 8761 con costo: 0.44831469655036926
Generacion: 8771 con costo: 0.44819721579551697
Generacion: 8781 con costo: 0.44779449701309204
Generacion: 8791 con costo: 0.448026567697525
Generacion: 8801 con costo: 0.44773054122924805
Generacion: 8811 con costo: 0.4476260542869568
Gen

...

Generacion: 191941 con costo: 0.1446976512670517
Generacion: 191951 con costo: 0.13971546292304993
Generacion: 191961 con costo: 0.14099743962287903
Generacion: 191971 con costo: 0.14824840426445007
Generacion: 191981 con costo: 0.16553309559822083
Generacion: 191991 con costo: 0.22069662809371948
Generacion: 192001 con costo: 0.29756227135658264
Generacion: 192011 con costo: 0.2582681179046631
Generacion: 192021 con costo: 0.20598788559436798
Generacion: 192031 con costo: 0.16044388711452484
Generacion: 192041 con costo: 0.1430843472480774
Generacion: 192051 con costo: 0.13982313871383667
Generacion: 192061 con costo: 0.14004971086978912
Generacion: 192071 con costo: 0.14149200916290283
Generacion: 192081 con costo: 0.14441905915737152
Generacion: 192091 con costo: 0.14676408469676971
Generacion: 192101 con costo: 0.1472972333431244
Generacion: 192111 con costo: 0.14608076214790344
Generacion: 192121 con costo: 0.1456882655620575
Generacion: 192131 con costo: 0.14622393250465393
Gener

Generacion: 193791 con costo: 0.13898056745529175
Generacion: 193801 con costo: 0.1453491449356079
Generacion: 193811 con costo: 0.16143949329853058
Generacion: 193821 con costo: 0.2176770269870758
Generacion: 193831 con costo: 0.32826298475265503
Generacion: 193841 con costo: 0.39262309670448303
Generacion: 193851 con costo: 0.5364066362380981
Generacion: 193861 con costo: 0.17304816842079163
Generacion: 193871 con costo: 0.142085000872612
Generacion: 193881 con costo: 0.14023976027965546
Generacion: 193891 con costo: 0.13970094919204712
Generacion: 193901 con costo: 0.1398991346359253
Generacion: 193911 con costo: 0.14007699489593506
Generacion: 193921 con costo: 0.1406906396150589
Generacion: 193931 con costo: 0.14187109470367432
Generacion: 193941 con costo: 0.14264045655727386
Generacion: 193951 con costo: 0.14395657181739807
Generacion: 193961 con costo: 0.1445615142583847
Generacion: 193971 con costo: 0.1447722166776657
Generacion: 193981 con costo: 0.1447620391845703
Generacion

Generacion: 195681 con costo: 0.15094396471977234
Generacion: 195691 con costo: 0.13959725201129913
Generacion: 195701 con costo: 0.13860489428043365
Generacion: 195711 con costo: 0.13859181106090546
Generacion: 195721 con costo: 0.1384580433368683
Generacion: 195731 con costo: 0.13947178423404694
Generacion: 195741 con costo: 0.14166560769081116
Generacion: 195751 con costo: 0.14602221548557281
Generacion: 195761 con costo: 0.14455953240394592
Generacion: 195771 con costo: 0.1412617415189743
Generacion: 195781 con costo: 0.14009179174900055
Generacion: 195791 con costo: 0.13996583223342896
Generacion: 195801 con costo: 0.14204666018486023
Generacion: 195811 con costo: 0.14361043274402618
Generacion: 195821 con costo: 0.14552532136440277
Generacion: 195831 con costo: 0.14611732959747314
Generacion: 195841 con costo: 0.14498375356197357
Generacion: 195851 con costo: 0.14301592111587524
Generacion: 195861 con costo: 0.14279823005199432
Generacion: 195871 con costo: 0.1453056037425995
Gen

Generacion: 197541 con costo: 0.1380976140499115
Generacion: 197551 con costo: 0.13692602515220642
Generacion: 197561 con costo: 0.13649067282676697
Generacion: 197571 con costo: 0.1373225897550583
Generacion: 197581 con costo: 0.13979053497314453
Generacion: 197591 con costo: 0.14255580306053162
Generacion: 197601 con costo: 0.14701689779758453
Generacion: 197611 con costo: 0.1497449278831482
Generacion: 197621 con costo: 0.152445986866951
Generacion: 197631 con costo: 0.15017761290073395
Generacion: 197641 con costo: 0.14164431393146515
Generacion: 197651 con costo: 0.13834720849990845
Generacion: 197661 con costo: 0.14007888734340668
Generacion: 197671 con costo: 0.14707978069782257
Generacion: 197681 con costo: 0.16103337705135345
Generacion: 197691 con costo: 0.18760845065116882
Generacion: 197701 con costo: 0.23255370557308197
Generacion: 197711 con costo: 0.30753859877586365
Generacion: 197721 con costo: 0.6144548654556274
Generacion: 197731 con costo: 0.3242371678352356
Generac

Generacion: 199391 con costo: 0.1354834884405136
Generacion: 199401 con costo: 0.1360791027545929
Generacion: 199411 con costo: 0.13954417407512665
Generacion: 199421 con costo: 0.14198368787765503
Generacion: 199431 con costo: 0.14155550301074982
Generacion: 199441 con costo: 0.13763199746608734
Generacion: 199451 con costo: 0.13511531054973602
Generacion: 199461 con costo: 0.13498149812221527
Generacion: 199471 con costo: 0.1354396939277649
Generacion: 199481 con costo: 0.136880561709404
Generacion: 199491 con costo: 0.13822375237941742
Generacion: 199501 con costo: 0.13973064720630646
Generacion: 199511 con costo: 0.14378371834754944
Generacion: 199521 con costo: 0.1466142237186432
Generacion: 199531 con costo: 0.1458388715982437
Generacion: 199541 con costo: 0.13982070982456207
Generacion: 199551 con costo: 0.13671043515205383
Generacion: 199561 con costo: 0.13777706027030945
Generacion: 199571 con costo: 0.14293348789215088
Generacion: 199581 con costo: 0.15283752977848053
Generac

In [ ]:
plt.plot(range(epochs), costos_totales)
plt.ylabel('Costos')
plt.xlabel('Generaciones')

In [96]:
#Predecir
predicciones = []

with torch.no_grad():
  for i, data in enumerate(X_test_t):
    y_pred_test = red_pytorch(data)
    predicciones.append(y_pred_test.argmax().item())

predicciones

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

In [97]:
#Evaluación de testeo
print(confusion_matrix(y_test, predicciones))
print(classification_report(y_test, predicciones))

[[127  23]
 [ 41  40]]
              precision    recall  f1-score   support

           0       0.76      0.85      0.80       150
           1       0.63      0.49      0.56        81

    accuracy                           0.72       231
   macro avg       0.70      0.67      0.68       231
weighted avg       0.71      0.72      0.71       231



###Crear Modelo con Keras/Tensorflow

In [ ]:
#importar modulo

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#Crear el modelo

red_keras = Sequential()
red_keras.add(Dense(8, input_dim = 8, activation= 'relu'))#entrada
red_keras.add(Dense(8, activation= 'relu'))
red_keras.add(Dense(4, activation= 'relu'))
red_keras.add(Dense(1, activation= 'sigmoid'))

red_keras.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [ ]:
#entrenamiento

historial = red_keras.fit(X_train, y_train, epochs = 2000, batch_size = 10, validation_data = (X_test, y_test))

In [ ]:
#Evaluación
evaluacion = red_keras.evaluate(X_test,y_test)
print("%s: %.2f%%" % (red_keras.metrics_names[1], evaluacion[1]*100))

In [ ]:
predicciones = red_keras.predict(X_test)
predicciones = np.argmax(predicciones, axis=1)

print(confusion_matrix(y_test, predicciones))
print(classification_report(y_test, predicciones))

In [ ]:
fig, ax = plt.subplots()
ax.plot(historial.history["loss"], 'r', marker='.', label = 'loss')
ax.plot(historial.history["val_loss"], 'b', marker='.', label = 'validación')
ax.legend();

In [ ]:
fig, ax = plt.subplots()
ax.plot(historial.history["accuracy"], 'r', marker='.', label = 'accuracy')
ax.plot(historial.history["val_accuracy"], 'b', marker='.', label = 'validación')
ax.legend();